In [1]:
import pandas as pd
import requests
import os

In [4]:
file_csv = '~/code/harlqeuinht/which_horse/raw_data/combined_flat2_csv.csv'

In [147]:
master_df = pd.read_csv(file_csv)

/tmp/ipykernel_134545/2751745270.py:1: DtypeWarning: Columns (22,53,62,71,80,89,98,107,116) have mixed types. Specify dtype option on import or set low_memory=False.
  master_df = pd.read_csv(file_csv)


In [164]:
df = master_df.sample(n=10)

In [165]:
df

,meeting_id,meeting_name,date,country_code,event_number,distance,distance_unit,distance_furlongs,distance_raw_furlongs,race_class,...,run_7_dsr,run_8_raw_post_race_rating_int,run_8_raw_post_race_rating_symbol,run_8_final_rating_int,run_8_race_type,run_8_race_class,run_8_race_class_normalised,run_8_track_type,run_8_win_lose,run_8_dsr
153239,28366,BATH,2021-08-18,GBR,6,1760,yards,8f,8.0,6,...,345.0,41.0,NaN,67.0,FLAT,6,Handicap,TURF,0.0,381.0
76123,9269,WOLVERHAMPTON,2020-06-21,GBR,3,1340,yards,6.1f,6.1,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
266024,37375,HAYDOCK,2023-07-22,GBR,2,1100,yards,5f,5.0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
228302,34577,KEMPTON,2022-11-02,GBR,7,2639,yards,1m4f,12.0,4,...,116.0,66.0,NaN,74.0,FLAT,4,Handicap,AWT,0.0,379.0
202256,33194,EPSOM,2022-06-30,GBR,1,2646,yards,1m4f,12.0,4,...,283.0,74.0,+,66.0,FLAT,4.0,Handicap,TURF,1.0,293.0
46434,8350,YORK,2019-09-08,GBR,1,1189,yards,5.4f,5.4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96473,10092,KEMPTON,2020-09-18,GBR,3,1540,yards,7f,7.0,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158207,28693,DONCASTER,2021-09-09,GBR,4,1760,yards,8f,8.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169986,29238,WOLVERHAMPTON,2021-11-08,GBR,5,1902,yards,1m0.5f,8.6,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167559,29104,CHELTENHAM,2021-10-23,GBR,7,3607,yards,2m0.5f,16.4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
columns_to_drop = df.filter(regex='[678]').columns
columns_to_drop = columns_to_drop.drop('bet365_odds')
df = df.drop(columns=(columns_to_drop))

Create a DataFrame which stores the meeting location name and long & lat. Note this has been done on the sample dataset, not the full dataset

### Locations API ###

In [137]:
def get_co_ordinates(df):
        # Create a list of all unique racecourse names
        location_names = sorted(df['meeting_name'].unique())
        # Clean racecourse names so they are reconisable by the geolocation API
        locations_df = pd.DataFrame(location_names, columns=['meeting_name'])
        locations_df['location_names_cleaned'] = locations_df['meeting_name'].replace({'BANGOR-ON-DEE':'BANGOR', 'NEWMARKET (JULY)':'NEWMARKET', ' ':'_'})
        locations_df['location_names_cleaned'] = locations_df['location_names_cleaned'].str.replace(' ', '_')

        # Iterate through the locations df, generating API endpoints for each row
        for index, location in enumerate(locations_df['location_names_cleaned']):
            base_url = 'https://maps.googleapis.com/maps/api/geocode/json?'
            api_key = os.environ['KEY']
            params = f'address={location}+racecourse&components=country:GB&key={api_key}'
            endpoint = f'{base_url}{params}'
            # Call the geolocation API, storing the results
            results = requests.get(endpoint).json()
            # Store the returned latitude and longitude data in the respective columns
            locations_df.loc[index, 'lat'] = results['results'][0]['geometry']['location']['lat']
            locations_df.loc[index, 'lng'] = results['results'][0]['geometry']['location']['lng']
        return locations_df

locations_df = get_co_ordinates(df)

In [153]:
def get_unique_races(df):
    unique_race_days_df = pd.DataFrame({'date': df['date'], 'meeting_name':df['meeting_name']}).drop_duplicates()
    return unique_race_days_df

unique_race_days_df = get_unique_races(test_df)
unique_race_days_df

,date,meeting_name
263863,2023-07-13,NEWMARKET
138765,2021-06-17,CHELMSFORD
72320,2020-06-09,HAYDOCK
253132,2023-05-26,BATH
108145,2020-11-17,NEWCASTLE
187333,2022-04-25,THIRSK
141856,2021-06-30,KEMPTON
70196,2020-03-10,NEWCASTLE
210429,2022-08-06,LINGFIELD
26189,2019-06-19,CHELMSFORD


In [154]:
unique_race_days_df = pd.merge(unique_race_days_df, locations_df, how='left', left_on='meeting_name', right_on='meeting_name')
unique_race_days_df

,date,meeting_name,location_names_cleaned,lat,lng
0,2023-07-13,NEWMARKET,NEWMARKET,52.238014,0.374746
1,2021-06-17,CHELMSFORD,CHELMSFORD,51.842047,0.512814
2,2020-06-09,HAYDOCK,HAYDOCK,53.480950,-2.627852
3,2023-05-26,BATH,BATH,51.418670,-2.407209
4,2020-11-17,NEWCASTLE,NEWCASTLE,55.031800,-1.611713
5,2022-04-25,THIRSK,THIRSK,54.231208,-1.351548
6,2021-06-30,KEMPTON,KEMPTON,51.417868,-0.406807
7,2020-03-10,NEWCASTLE,NEWCASTLE,55.031800,-1.611713
8,2022-08-06,LINGFIELD,LINGFIELD,51.169842,-0.006314
9,2019-06-19,CHELMSFORD,CHELMSFORD,51.842047,0.512814


In [155]:
def generate_endpoint(row):
        base_url = 'https://archive-api.open-meteo.com/v1/archive?'
        latitude = row['lat']
        longitude = row['lng']
        date = row['date']
        params = '&daily=temperature_2m_mean,precipitation_sum,wind_speed_10m_max&wind_speed_unit=mph'
        return f'{base_url}&latitude={latitude}&longitude={longitude}&start_date={date}&end_date={date}&{params}'

unique_race_days_df['endpoint'] = unique_race_days_df.apply(generate_endpoint, axis=1)

In [156]:
unique_race_days_df.head()

,date,meeting_name,location_names_cleaned,lat,lng,endpoint
0,2023-07-13,NEWMARKET,NEWMARKET,52.238014,0.374746,https://archive-api.open-meteo.com/v1/archive?...
1,2021-06-17,CHELMSFORD,CHELMSFORD,51.842047,0.512814,https://archive-api.open-meteo.com/v1/archive?...
2,2020-06-09,HAYDOCK,HAYDOCK,53.480950,-2.627852,https://archive-api.open-meteo.com/v1/archive?...
3,2023-05-26,BATH,BATH,51.418670,-2.407209,https://archive-api.open-meteo.com/v1/archive?...
4,2020-11-17,NEWCASTLE,NEWCASTLE,55.031800,-1.611713,https://archive-api.open-meteo.com/v1/archive?...


In [157]:
def call_weather_api(row):
        response = requests.get(row['endpoint'])
        data = response.json()

        temp = data['daily']['temperature_2m_mean'][0]
        precipitation = data['daily']['precipitation_sum'][0]
        wind = data['daily']['wind_speed_10m_max'][0]
        return temp, precipitation, wind

unique_race_days_df[['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max']] = unique_race_days_df.apply(call_weather_api, axis=1, result_type='expand')

In [158]:
unique_race_days_df.head()

,date,meeting_name,location_names_cleaned,lat,lng,endpoint,temperature_2m_mean,precipitation_sum,wind_speed_10m_max
0,2023-07-13,NEWMARKET,NEWMARKET,52.238014,0.374746,https://archive-api.open-meteo.com/v1/archive?...,17.2,1.1,11.2
1,2021-06-17,CHELMSFORD,CHELMSFORD,51.842047,0.512814,https://archive-api.open-meteo.com/v1/archive?...,18.9,11.7,9.6
2,2020-06-09,HAYDOCK,HAYDOCK,53.480950,-2.627852,https://archive-api.open-meteo.com/v1/archive?...,12.2,1.2,9.2
3,2023-05-26,BATH,BATH,51.418670,-2.407209,https://archive-api.open-meteo.com/v1/archive?...,12.6,0.0,11.5
4,2020-11-17,NEWCASTLE,NEWCASTLE,55.031800,-1.611713,https://archive-api.open-meteo.com/v1/archive?...,14.1,0.2,16.9


### ALTOGETHER NOW ###

In [170]:
def get_weather_final(df):
    def get_co_ordinates(df):
            location_names = sorted(df['meeting_name'].unique())
            locations_df = pd.DataFrame(location_names, columns=['meeting_name'])
            locations_df['location_names_cleaned'] = locations_df['meeting_name'].replace({'BANGOR-ON-DEE':'BANGOR', 'NEWMARKET (JULY)':'NEWMARKET', ' ':'_'})
            locations_df['location_names_cleaned'] = locations_df['location_names_cleaned'].str.replace(' ', '_')

            for index, location in enumerate(locations_df['location_names_cleaned']):
                base_url = 'https://maps.googleapis.com/maps/api/geocode/json?'
                api_key = os.environ['KEY']
                params = f'address={location}+racecourse&components=country:GB&key={api_key}'
                endpoint = f'{base_url}{params}'
                # Call the geolocation API, storing the results
                results = requests.get(endpoint).json()
                # Store the returned latitude and longitude data in the respective columns
                locations_df.loc[index, 'lat'] = results['results'][0]['geometry']['location']['lat']
                locations_df.loc[index, 'lng'] = results['results'][0]['geometry']['location']['lng']
            return locations_df

    locations_df = get_co_ordinates(df)


    def get_unique_races(df):
        unique_race_days_df = pd.DataFrame({'date': df['date'], 'meeting_name':df['meeting_name']}).drop_duplicates()
        return unique_race_days_df

    unique_race_days_df = get_unique_races(df)
    unique_race_days_df = pd.merge(unique_race_days_df, locations_df, how='left', left_on='meeting_name', right_on='meeting_name')

    def generate_endpoint(row):
            base_url = 'https://archive-api.open-meteo.com/v1/archive?'
            latitude = row['lat']
            longitude = row['lng']
            date = row['date']
            params = '&daily=temperature_2m_mean,precipitation_sum,wind_speed_10m_max&wind_speed_unit=mph'
            return f'{base_url}&latitude={latitude}&longitude={longitude}&start_date={date}&end_date={date}&{params}'

    unique_race_days_df['endpoint'] = unique_race_days_df.apply(generate_endpoint, axis=1)

    def call_weather_api(row):
            response = requests.get(row['endpoint'])
            data = response.json()

            temp = data['daily']['temperature_2m_mean'][0]
            precipitation = data['daily']['precipitation_sum'][0]
            wind = data['daily']['wind_speed_10m_max'][0]
            return temp, precipitation, wind

    unique_race_days_df[['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max']] = unique_race_days_df.apply(call_weather_api, axis=1, result_type='expand')

    updated_df = pd.merge(df, unique_race_days_df, on=['date', 'meeting_name'], how='left')
    updated_df = updated_df.drop(columns=['endpoint', 'lat', 'lng', 'location_names_cleaned'])
    return updated_df

In [171]:
get_weather_final(df)

,meeting_id,meeting_name,date,country_code,event_number,distance,distance_unit,distance_furlongs,distance_raw_furlongs,race_class,...,run_8_final_rating_int,run_8_race_type,run_8_race_class,run_8_race_class_normalised,run_8_track_type,run_8_win_lose,run_8_dsr,temperature_2m_mean,precipitation_sum,wind_speed_10m_max
0,28366,BATH,2021-08-18,GBR,6,1760,yards,8f,8.0,6,...,67.0,FLAT,6,Handicap,TURF,0.0,381.0,14.7,0.0,14.8
1,9269,WOLVERHAMPTON,2020-06-21,GBR,3,1340,yards,6.1f,6.1,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.8,6.9,16.0
2,37375,HAYDOCK,2023-07-22,GBR,2,1100,yards,5f,5.0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.2,21.4,15.2
3,34577,KEMPTON,2022-11-02,GBR,7,2639,yards,1m4f,12.0,4,...,74.0,FLAT,4,Handicap,AWT,0.0,379.0,12.2,8.0,22.4
4,33194,EPSOM,2022-06-30,GBR,1,2646,yards,1m4f,12.0,4,...,66.0,FLAT,4.0,Handicap,TURF,1.0,293.0,14.1,1.8,10.9
5,8350,YORK,2019-09-08,GBR,1,1189,yards,5.4f,5.4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.5,0.0,9.6
6,10092,KEMPTON,2020-09-18,GBR,3,1540,yards,7f,7.0,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.6,0.0,16.0
7,28693,DONCASTER,2021-09-09,GBR,4,1760,yards,8f,8.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.8,0.5,10.0
8,29238,WOLVERHAMPTON,2021-11-08,GBR,5,1902,yards,1m0.5f,8.6,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.2,0.2,10.0
9,29104,CHELTENHAM,2021-10-23,GBR,7,3607,yards,2m0.5f,16.4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.5,0.0,15.6


In [ ]:
def create_weather_api_endpoint(df):
    def generate_endpoint(row):
        base_url = 'https://archive-api.open-meteo.com/v1/archive?'
        latitude = row['lat']
        longitude = row['lng']
        date = row['race_dates']
        params = '&daily=temperature_2m_mean,precipitation_sum,wind_speed_10m_max&wind_speed_unit=mph'
        return f'{base_url}&latitude={latitude}&longitude={longitude}&start_date={date}&end_date={date}&{params}'

    df['endpoint'] = df.apply(generate_endpoint, axis=1)



    # Create a function to call the weather API for each row in the unique_race_days_df
    def call_weather_api(row):
        response = requests.get(row['endpoint'])
        data = response.json()

        temp = data['daily']['temperature_2m_mean'][0]
        precipitation = data['daily']['precipitation_sum'][0]
        wind = data['daily']['wind_speed_10m_max'][0]
        return temp, precipitation, wind

    unique_race_days_df[['temp', 'precipitation', 'wind']] = unique_race_days_df.apply(call_weather_api, axis=1, result_type='expand')


    return unique_race_days_df

create_weather_api_endpoint(unique_race_days_df)

In [ ]:
def get_raceday_weather(test_df):
    a = get_co_ordinates(test_df)
    b = get_weather_data(a)
    c =


In [192]:
location_names = sorted(df['meeting_name'].unique())
#location_names = [location for location in location_names if location.lower() != 'newmarket (july)']
locations_df = pd.DataFrame(location_names, columns=['meeting_name'])
locations_df['location_names_cleaned'] = locations_df['meeting_name'].replace({'BANGOR-ON-DEE':'BANGOR', 'NEWMARKET (JULY)':'NEWMARKET', ' ':'_'})
locations_df['location_names_cleaned'] = locations_df['location_names_cleaned'].str.replace(' ', '_')

Temporarily reduce the dataframe to 5 rows

In [83]:
locations_df.head()

,meeting_name,location_names_cleaned,lat,lng
0,AINTREE,AINTREE,53.473476,-2.954128
1,ASCOT,ASCOT,51.412167,-0.679485
2,AYR,AYR,55.467721,-4.612545
3,BANGOR-ON-DEE,BANGOR,53.004035,-2.909421
4,BATH,BATH,51.418670,-2.407209


Create API endpoint URLs

In [198]:
api_urls = []
all_results = []
for index, location in enumerate(locations_df['location_names_cleaned']):
    base_url = 'https://maps.googleapis.com/maps/api/geocode/json?'
    api_key = os.environ['KEY']
    params = f'address={location}+racecourse&components=country:GB&key={api_key}'
    endpoint = f'{base_url}{params}'
    results = requests.get(endpoint).json()
    results_dict = {'location':location,
                    'co_ords':{'lat':results['results'][0]['geometry']['location']['lat'],
                               'lng':results['results'][0]['geometry']['location']['lng']
                              }
                   }
    all_results.append(results_dict)
    locations_df.loc[index, 'lat'] = results['results'][0]['geometry']['location']['lat']
    locations_df.loc[index, 'lng'] = results['results'][0]['geometry']['location']['lng']


In [84]:
locations_df.head()

,meeting_name,location_names_cleaned,lat,lng
0,AINTREE,AINTREE,53.473476,-2.954128
1,ASCOT,ASCOT,51.412167,-0.679485
2,AYR,AYR,55.467721,-4.612545
3,BANGOR-ON-DEE,BANGOR,53.004035,-2.909421
4,BATH,BATH,51.418670,-2.407209


### Weather API ###

In [200]:
sample_df = df.head(5)
sample_df

,meeting_id,meeting_name,date,country_code,event_number,distance,distance_unit,distance_furlongs,distance_raw_furlongs,race_class,...,run_4_dsr,run_5_raw_post_race_rating_int,run_5_raw_post_race_rating_symbol,run_5_final_rating_int,run_5_race_type,run_5_race_class,run_5_race_class_normalised,run_5_track_type,run_5_win_lose,run_5_dsr
0,3668,AYR,2020-07-20,GBR,1,1100,yards,5f,5.0,6,...,305.0,59.0,NaN,66.0,FLAT,5,Handicap,TURF,0.0,342.0
1,3668,AYR,2020-07-20,GBR,7,1760,yards,8f,8.0,6,...,226.0,60.0,NaN,71.0,FLAT,6,Handicap,TURF,0.0,279.0
2,3668,AYR,2020-07-20,GBR,7,1760,yards,8f,8.0,6,...,371.0,56.0,NaN,68.0,FLAT,3,Handicap,TURF,0.0,383.0
3,3668,AYR,2020-07-20,GBR,7,1760,yards,8f,8.0,6,...,47.0,59.0,NaN,69.0,FLAT,6,Handicap,TURF,0.0,272.0
4,3668,AYR,2020-07-20,GBR,7,1760,yards,8f,8.0,6,...,417.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
df = pd.merge(df,locations_df, on = 'meeting_name')
df

,meeting_id,meeting_name,date,country_code,event_number,distance,distance_unit,distance_furlongs,distance_raw_furlongs,race_class,...,run_5_final_rating_int,run_5_race_type,run_5_race_class,run_5_race_class_normalised,run_5_track_type,run_5_win_lose,run_5_dsr,location_names_cleaned,lat,lng
0,3668,AYR,2020-07-20,GBR,1,1100,yards,5f,5.0,6,...,66.0,FLAT,5,Handicap,TURF,0.0,342.0,AYR,55.467721,-4.612545
1,3668,AYR,2020-07-20,GBR,7,1760,yards,8f,8.0,6,...,71.0,FLAT,6,Handicap,TURF,0.0,279.0,AYR,55.467721,-4.612545
2,3668,AYR,2020-07-20,GBR,7,1760,yards,8f,8.0,6,...,68.0,FLAT,3,Handicap,TURF,0.0,383.0,AYR,55.467721,-4.612545
3,3668,AYR,2020-07-20,GBR,7,1760,yards,8f,8.0,6,...,69.0,FLAT,6,Handicap,TURF,0.0,272.0,AYR,55.467721,-4.612545
4,3668,AYR,2020-07-20,GBR,7,1760,yards,8f,8.0,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AYR,55.467721,-4.612545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282001,37996,NEWMARKET,2023-09-28,GBR,3,1320,yards,6f,6.0,2,...,81.0,FLAT,1.0,Listed,TURF,0.0,145.0,NEWMARKET,52.238014,0.374746
282002,37996,NEWMARKET,2023-09-28,GBR,2,1760,yards,8f,8.0,4,...,70.0,FLAT,5.0,Novice,TURF,0.0,105.0,NEWMARKET,52.238014,0.374746
282003,37996,NEWMARKET,2023-09-28,GBR,2,1760,yards,8f,8.0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NEWMARKET,52.238014,0.374746
282004,37996,NEWMARKET,2023-09-28,GBR,3,1320,yards,6f,6.0,2,...,105.0,FLAT,1.0,Group 3,TURF,0.0,392.0,NEWMARKET,52.238014,0.374746


In [ ]:
for index, row in sample_df.iterrows():
    lat = sample_df['lat']
    lng = sample_df['lng']
    date = sample_df['date']

In [89]:
weather_temp = []
weather_rain = []
weather_wind = []

all_results = []
for index, location in enumerate(locations_df['location_names_cleaned']):
    base_url = 'https://archive-api.open-meteo.com/v1/archive?'
    params = 'daily=temperature_2m_mean,precipitation_sum,rain_sum,wind_speed_10m_max,wind_direction_10m_dominant'
    endpoint = f'{base_url}{params}'
    results = requests.get(endpoint).json()
    results_dict = {'location':location,
                    'co_ords':{'lat':results['results'][0]['geometry']['location']['lat'],
                               'lng':results['results'][0]['geometry']['location']['lng']
                              }
                   }
    all_results.append(results_dict)
    locations_df.loc[index, 'lat'] = results['results'][0]['geometry']['location']['lat']
    locations_df.loc[index, 'lng'] = results['results'][0]['geometry']['location']['lng']


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
base_url = 'https://archive-api.open-meteo.com/v1/archive?'
longitude = '-1.597'
latitude = '52.279'
start_date = '2020-01-01'
end_date = '2020-01-07'
params = 'daily=temperature_2m_mean,precipitation_sum,rain_sum,wind_speed_10m_max,wind_direction_10m_dominant'
endpoint = f'{base_url}&latitude={latitude}&longitude={longitude}&start_date={start_date}&end_date={end_date}&{params}'
endpoint

print(requests.get(endpoint).json())

In [179]:
weather_result = requests.get(endpoint).json()
precipitation = weather_result['daily']['precipitation_sum']
wind = weather_result['daily']['wind_speed_10m_max']
temp = weather_result['daily']['temperature_2m_mean']

In [160]:
base_url = 'https://archive-api.open-meteo.com/v1/archive?'
longitude = '-1.597'
latitude = '52.279'
start_date = '2020-01-01'
end_date = '2020-01-07'
params = 'daily=temperature_2m_mean,precipitation_sum,rain_sum,wind_speed_10m_max,wind_direction_10m_dominant'
endpoint = f'{base_url}&latitude={latitude}&longitude={longitude}&start_date={start_date}&end_date={end_date}&{params}'
endpoint

print(requests.get(endpoint).json())


{'latitude': 52.26713, 'longitude': -1.6636047, 'generationtime_ms': 0.33104419708251953, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 51.0, 'daily_units': {'time': 'iso8601', 'temperature_2m_mean': '°C', 'precipitation_sum': 'mm', 'rain_sum': 'mm', 'wind_speed_10m_max': 'km/h', 'wind_direction_10m_dominant': '°'}, 'daily': {'time': ['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04', '2020-01-05', '2020-01-06', '2020-01-07'], 'temperature_2m_mean': [5.9, 8.3, 7.0, 5.8, 6.1, 6.9, 9.3], 'precipitation_sum': [0.0, 0.0, 4.8, 0.0, 0.0, 1.4, 0.1], 'rain_sum': [0.0, 0.0, 4.8, 0.0, 0.0, 1.4, 0.1], 'wind_speed_10m_max': [15.3, 27.1, 23.0, 27.3, 22.7, 28.3, 31.3], 'wind_direction_10m_dominant': [171, 204, 280, 267, 222, 211, 220]}}


In [161]:
endpoint

'https://archive-api.open-meteo.com/v1/archive?&latitude=52.279&longitude=-1.597&start_date=2020-01-01&end_date=2020-01-07&daily=temperature_2m_mean,precipitation_sum,rain_sum,wind_speed_10m_max,wind_direction_10m_dominant'

In [58]:
latitude = locations_df['latitude']
longitude = locations_df['longitude']

KeyError: 'latitude'

In [ ]:
# This code works and returns a 200 code for current weather data
response = requests.get('http://api.weatherapi.com/v1/current.json?key=581fec608fba4a5699790722240703&dt=2020-01-01&q=London')
response

In [ ]:
print(response.json())

In [3]:
os.environ['KEY']

'yscbfocbiybdiuhabpiusxhb'